In [1]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
import random
from scipy import stats
import random
import pywt.data
from PIL import Image
import pandas as pd
import seaborn as sns
import os
import pickle

In [3]:

# assign directory
directory = '/Users/yashd/Desktop/hierarchical-bayesian-model-validation/Data/Panoptic Agriculture/Original 400 Images'
 
# iterate over files in
# that directory
file_list = [os.path.join(directory, filename) for filename in os.listdir(directory)]
file_names = os.listdir(directory)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/Panoptic Agriculture/Cleaned Dataset'

In [7]:
df_dict = dict()
image = Image.open(file_list[0]).convert('L')
first_image = pywt.wavedec2(image, 'db1')
layer_len = len(first_image)
print(str(layer_len) + " layers being used")
for i in range(layer_len):
        df = pd.DataFrame(columns=["Image ID", "Orientation", "Data"])
        #df = pd.DataFrame(columns=["Image ID", "Orientation", "Data", "Flattened Data"])
        df_dict[i+1] = df

8 layers being used


In [8]:
for k in range(len(file_list)):
    image = Image.open(file_list[k]).convert('L')
    image = np.array(image)
   
    name = file_names[k].split(".")[0]
    transformed = pywt.wavedec2(image, 'db1')
    #df_dict[1].loc[len(df_dict[1].index)] = [name, "ONELAYER", np.array(transformed[0][0]), np.array(transformed[0][0]).flatten()]
    df_dict[1].loc[len(df_dict[1].index)] = [name, "ONELAYER", np.array(transformed[0][0])]
    direction_names = ['Horizontal detail', 'Vertical detail', 'Diagonal detail']

    for i in range(1, layer_len): 
        for j in range(len(transformed[i])):
            arr = np.array(transformed[i][j])
            #df_dict[i+1].loc[len(df_dict[i+1].index)] = [name, direction_names[j], arr, arr.flatten()]
            df_dict[i+1].loc[len(df_dict[i+1].index)] = [name, direction_names[j], arr]


In [9]:
def convert_to_wavelet_basis(folder_dir,  normalized = True, basis="db1"):
    file_list = [os.path.join(folder_dir, filename) for filename in os.listdir(folder_dir)]
    file_names = os.listdir(folder_dir)
    #Setup df Dict
    df_dict = dict()
    image = Image.open(file_list[0]).convert('L')
    first_image = pywt.wavedec2(image, basis)
    layer_len = len(first_image)
    print(str(layer_len) + " layers being used")
    for i in range(layer_len):
        #df = pd.DataFrame(columns=["Image ID", "Orientation", "Data", "Flattened Data"])
        df = pd.DataFrame(columns=["Image ID", "Orientation", "Data"])
        df_dict[i+1] = df
    
    
    #Fill DF DICT
    for k in range(len(file_list)):
        image = Image.open(file_list[k]).convert('L')
        image = np.array(image)
        if normalized:
            std= np.std(image)
            mean = np.mean(image)
            image = (image- mean)/std 
            #image = image * 255
            
        name = file_names[k].split(".")[0]
        transformed = pywt.wavedec2(image, basis)
        #df_dict[1].loc[len(df_dict[1].index)] = [name, "ONELAYER", np.array(transformed[0][0]), np.array(transformed[0][0]).flatten()]
        df_dict[1].loc[len(df_dict[1].index)] = [name, "ONELAYER", np.array(transformed[0][0])]
        direction_names = ['Horizontal detail', 'Vertical detail', 'Diagonal detail']

        for i in range(1, layer_len): 
            for j in range(len(transformed[i])):
                arr = np.array(transformed[i][j])
                df_dict[i+1].loc[len(df_dict[i+1].index)] = [name, direction_names[j], arr]
                #df_dict[i+1].loc[len(df_dict[i+1].index)] = [name, direction_names[j], arr.flatten()]

    return df_dict

In [10]:
converted = convert_to_wavelet_basis(directory, normalized = False)

8 layers being used


In [11]:
converted[4]

,Image ID,Orientation,Data
0,test280,Horizontal detail,"[[-26.062500000000057, 110.53125000000006, -35..."
1,test280,Vertical detail,"[[437.81250000000057, -17.031249999999886, -16..."
2,test280,Diagonal detail,"[[-103.56249999999997, 158.21875000000006, 205..."
3,test294,Horizontal detail,"[[-226.12499999999994, -114.18750000000016, 26..."
4,test294,Vertical detail,"[[729.4375000000014, 24.5625, -19.062500000000..."
...,...,...,...
1198,test266,Vertical detail,"[[17.812500000000455, 105.96875000000023, -156..."
1199,test266,Diagonal detail,"[[203.49999999999997, 130.09374999999997, -183..."
1200,test299,Horizontal detail,"[[-32.906250000000014, 70.375, 99.062500000000..."
1201,test299,Vertical detail,"[[-7.343749999999943, -83.75000000000006, -7.1..."


In [12]:
def dict_to_pickle(converted_directory, converted, name):
    filename = name
    filename = os.path.join(converted_directory, filename)
    #converted[keys].to_csv(filename, sep=',', index=False, encoding='utf-8')
    with open(filename+".pickle", 'wb') as handle:
        pickle.dump(converted, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
norm_converted_directory = "/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/Wavelet Transform/Converted Panoptic Agricultural Data"
norm_converted = convert_to_wavelet_basis(directory, normalized = True)


8 layers being used


In [15]:
dict_to_pickle(norm_converted_directory, norm_converted, "Toy_Data_Dict_Normalized")